# Imports

In [5]:
from typing import List, Dict, Tuple, Set

import requests
from bs4 import BeautifulSoup
from pathlib import Path

from datetime import date
import pandas as pd
import os
import numpy as np

from tqdm import tqdm # https://github.com/tqdm/tqdm#ipython-jupyter-integration
# For retries.
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import time
from collections import deque

# Local imports.
import constants as c

# Function definitions

In [2]:
class WebCrawler():
    """ This class performs a variety of crawling tasks. """

    def __init__(self):
        pass

    def crawl_href_links_on_webpage(
            self, 
            url: str, 
            website_contains_content_all: str = [], 
            url_shall_contain_all: List[str] = [], 
            url_shall_contain_some: List[str] = [],
            url_must_not_contain_any: List[str] = [],
            forbidden_url_prefixes: List[str] = [], 
            forbidden_url_postfixes: List[str] = [], 
            reconnection_trys: int = 3,
            search_recursively: bool = False,
            link_depth: int = 0,
            verbose: bool = False
        ) -> Tuple[List[str], List[str]]:
        """ Crawls the webpage with the given url and collects all links on it.
            Use 'content_keyword' to demand that the links point to websites containing this keyword.
            Use 'url_shall_contain' or 'url_must_not_contain' to demant that the links contain or do not contain the keyword in their url.
            Use 'forbidden_url_prefixes' or 'forbidden_url_postfixes' to exclude urls with specific starts or endings.
            Use 'recursive_search' to allow crawling the found links as well. The implementation is not recursive (to avoid opening many crawler sessions) but the search is.
            # TODO: TRY RECURISVE (no deque needed)
            This function also maintains the lists of accepted and discarded links of the crawler class.            
            It returns the newly found accepted links, and the entire list of accepted links.

            Example usage:
            - Pass a tournament site and find all TopTurnier competition sites on it.
            - Pass a website listing dance clubs and find all websites to dance clubs on it.
        """
        def check_url(u: str) -> bool:
            """ Test the given url, if it fulfills the set url critera. 
                Returns the decision.
            """            
            contains_some_desired   = True
            if len(url_shall_contain_some) > 0:
                contains_some_desired = any([s in u for s in url_shall_contain_some])
            
            contains_all_desired    = all([s in u for s in url_shall_contain_all])
            contains_none_forbidden = not any([s in u for s in url_must_not_contain_any])
            allowed_url_prefix      = not u.startswith(tuple(forbidden_url_prefixes))
            allowed_url_postfix     = not u.endswith(tuple(forbidden_url_postfixes))
            return contains_all_desired and contains_some_desired and contains_none_forbidden and allowed_url_prefix and allowed_url_postfix

        def check_webpage_content(u) -> bool:
            # If no desired content is specified, it is accepted by default.
            if len(website_contains_content_all) == 0:
                return True
            try:
                source_code = session.get(u)                    
                link_soup = BeautifulSoup(source_code.text, "html.parser")

                # At this point it is already assumed that the link shall be saved. Negate this, if a content keyword is missing.
                return all([s in str(link_soup) for s in website_contains_content_all])
                                
            except Exception as e:
                print(f"Error: On site '{url}', could not process link '{u}'\n>>> {e}")
                time.sleep(.2)
                return False

        link_depth: int = 0

        # Organize all anchors (all local URLs) in a queue.
        webpage_search_queue = deque()
        webpage_search_queue.append((url, link_depth))
        searched_links_and_depth: Dict[str, int] = {}

        # Collect the targeted links. Also 
        target_links: List[str] = []
                
        # In case the host rejects to many calls, set up retries.
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=1.0)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount("https://", adapter)

        reconnection_ctr: int = reconnection_trys
        start_time = time.time()

        while len(webpage_search_queue) > 0 and not (reconnection_ctr < 0):
            # Get the next webpage to be searched.
            webpage, depth = webpage_search_queue.pop()
            # Do not process this link, if the link has been discoverd by clicking through already 'link_depth' many links.
            if depth > link_depth: continue

            # Try reading the webpage.
            try:                
                # Extract the source code.
                source_code = session.get(webpage)
                # Read the source code.
                soup = BeautifulSoup(source_code.text, "html.parser")

                # Iterate through all anchors on this website. Determine in a second step if they are hyperreferences.
                local_anchors = list(set(soup.find_all("a")))
                for local_anchor in local_anchors:
                    # Check for basic suitability of the link.
                    link_has_no_href: bool = "href" not in local_anchor.attrs
                    link_is_download: bool = local_anchor.attrs.get("class") == 'download-link'
                    link_is_file: bool     = Path(local_link).suffix == ""

                    if link_has_no_href or link_is_download or link_is_file: continue
                    
                    # Extract the actual link.
                    local_link: str = local_anchor.attrs["href"]
                    local_link_depth: int = depth + 1
                    if verbose: print("\r" + " " * 150 + f"\rLink: '{local_link}'", end="")
                   
                    # First, if the link is already known, do not process it twice.
                    # This behaviour changes if recursive search is on, and the link was now reached quicker than before.
                    if local_link in searched_links_and_depth.keys():
                        if verbose: print(f" => Already searched", end="")
                        # If this link was now found again, but with less clicks than before, 
                        # and recursive search is on, allow searching it again.
                        _, prev_depth = searched_links_and_depth.get(local_link)
                        if search_recursively and prev_depth > local_link_depth:
                            # Forget that the link is already known and search it again.
                            searched_links_and_depth.pop(local_link)
                        else: 
                            # Otherwise continue with the next link since this link has already been processed.
                            continue
                    
                    # Add the link to the already searched ones.
                    searched_links_and_depth[local_link] = local_link_depth
                    # If the recursive search is activated, add the link to the search queue.
                    if search_recursively:
                        webpage_search_queue.appendleft((local_link, local_link_depth))

                    # Now check if the link is a target link.
                    link_url_is_accepted: bool = False
                    link_content_is_accepted: bool = False

                    # Test the link for the desired url features.
                    link_url_is_accepted = check_url(local_link)
                    if verbose: print(f" => Desired url", end="")
                                            
                    # Test if the content of the links website is as desired.
                    # Therefore first crawl content of the linked website.
                    link_content_is_accepted = check_webpage_content(local_link)
                    if verbose: print(f" => {'D' if link_content_is_accepted else 'Und'}esired content", end="")

                    # Save the link.
                    if link_url_is_accepted and link_content_is_accepted:
                        target_links.append(local_link)
                        
                # Update the connection counter such that the while loop ends after this succesful connection.
                reconnection_ctr = -2

            except Exception as e:
                print(f"Error: Could not completely process site '{url}'\n>>>{e}")
                time.sleep(1)
                # Decrement the connection counter and try once more.
                reconnection_ctr -= 1
                    
            if verbose: print()
        
        runtime = time.time() - start_time
        print(f"\nLinks found: {len(target_links)}\tAccepted {len(target_links)}")
        print(f"Runtime: {runtime: 0.2f} sec\t({runtime / max(1, len(target_links)): 0.2f} sec per link, {runtime / max(1, len(target_links)): 0.2f} sec per accepted link)")

        return target_links

    def crawl_all_local_links_to_website(
            self, 
            url: str, 
            reconnection_trys: int = 3,
            link_depth: int = 2
        ) -> List[str]:
        """ Returns a list of all easily accessable links (click-depth 'link_depth') within the same website. 
            Find them by recursively following links on the given startpage.
            Links with suffixes (like '.jpg' or '.pdf') are ignored.

            Example Usage:
            - Collect all links on a club site. In a next step these can be investigated to be tournament links.

        Args:
            start_anchor (str, optional): URL anchor to start constructing new subsites. Defaults to "/".
            recusrion_depth (int, optional): Depth on how many URLs will be followed in order to find target sites. Defaults to 5.
            verbose (bool, optional): If True - print more statements will be printed to the console. Defaults to False.

        Returns:
            List[str]: A list of found URLs.
        """
        # Initial print to offset '\r'.
        print()
        
        depth: int = 0
        local_links: List[str] = []

        # Organize all anchors (all local URLs) in a queue.
        webpage_queue = deque()
        webpage_queue.append((url, depth))

        # Assemble the first url to search.
        # All urls will be build from the base url. This does not allow to jump to other websites during the recursive crawl, 
        # but allows to find links to them.
        url_depth_dict: Dict[str, int] = {url: 1}

        # In case the host rejects to many calls, set up retries.
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=1.0)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount("https://", adapter)

        reconnection_ctr: int = reconnection_trys
        start_time = time.time()
        
        # While the queue is not empty and the connection holds, crawl for urls.
        while len(webpage_queue) > 0 and reconnection_ctr > 0:
            # Fetch the next webpage to be searched.
            webpage, depth = webpage_queue.pop()
            # Do not continue, if the reached link has been discoverd by clicking through 'link_depth' many links.
            if depth > link_depth:
                continue
            try:
                # Extract the source code.
                source_code = session.get(webpage)
                # Read the source code.
                soup = BeautifulSoup(source_code.text, "html.parser")
                # Find all hyperreferences (html tag <"a href">).
                local_links = [link.attrs["href"] for link in soup.find_all("a") if "href" in link.attrs]

                for local_link in local_links:
                    # Look for links that start with the base url.
                    link_is_not_local: bool = not local_link.startswith(self.base_url)
                    # For the next search iteration, only use anchors, which do not link to a file - and thus do not have a suffix.
                    # Like for example '.html', '.pdf',...
                    link_is_file: bool = Path(local_link).suffix == ""

                    if link_is_not_local or not link_is_file:
                        continue
            
                    # Reconstruct the current local anchor
                    local_anchor: str = local_link.replace(self.base_url, "")

                    # Also do not search already known URLs twice, if not a shorter link chain was found.
                    if local_link in url_depth_dict.keys():
                        _, prev_depth = url_depth_dict.get(local_link)
                        if prev_depth > depth + 1:
                            url_depth_dict[local_link] = depth + 1
                            webpage_queue.appendleft((local_anchor, depth + 1))
                        continue
                    else:
                        webpage_queue.appendleft((local_anchor, depth + 1))

                    url_depth_dict[local_link] = depth + 1
                    local_links.append(local_link)

                print(f"\rUrls found: {len(url_depth_dict)}\tPending: {len(webpage_queue)}\tCurrent anchor: {webpage}", end=" "*50)
            except Exception as e:
                print(e)
                print(f"Error while searching in '{self.base_url + webpage}':\n{e}")
                time.sleep(2)
                reconnection_ctr -= 1

        runtime = time.time() - start_time
        print(f"\nFound {len(local_links)} new subsites in {runtime: 0.2f} sec\t({runtime / max(1, len(local_links)): 0.2f} sec per url)")
        print(f"Number of known subsites in total {len(url_depth_dict)}")
        
        return list(url_depth_dict.keys())


# Usage

## 1. Crawl for club names
Given a site that contains links to clubs, crawl links that may lead to them. These are links that do not contain the original site name in the url.

In [3]:
# Crawl for club names using the club finder table.
find_clubs = pd.read_csv(f"{c.DIR_RESULTS}/{c.FilenameList.FIND_CLUBS}")
# for i, row in find_clubs.iterrows():
row = find_clubs.iloc[0]
site = row.iloc[0]
federal_state = row.iloc[1]
crawl_date = row.iloc[2]
wc = WebCrawler(site)

In [4]:
# Since we are looking for links to other club sites, these urls must not contain the site name of the current url.
new_club_sites, old_club_sites = wc.crawl_href_links_on_webpage(
    url=site, 
    url_must_not_contain_any=[c.get_site_name_from_url(site)], 
    forbidden_url_prefixes=c.ClubsDf.BAD_LINK_PREFIXES, 
    forbidden_url_postfixes=c.ClubsDf.BAD_LINK_POSTFIXES
)
print(f"Saved {len(old_club_sites)} valid links ({len(new_club_sites)} new ones) from site '{site}'")

Saved 174 valid links (174 new ones) from site 'https://tnw.de/verband/vereine/'


## 2. Go through the crawled club names
Try to find a collection of tournaments, if found, save the site to the clubs list and the original site.

In [5]:
club_site = 'https://www.tanzsportkreis-sankt-augustin.de/'
# old_club_sites[0]
wc = WebCrawler(club_site)

In [6]:
new_links, all_links = wc.crawl_href_links_on_webpage(
    url=club_site,
    url_shall_contain_some=['urnier', 'rgebnisse', 'ompetition', 'esults'],
    forbidden_url_prefixes=c.ClubsDf.BAD_LINK_PREFIXES,
    forbidden_url_postfixes=c.ClubsDf.BAD_LINK_POSTFIXES,
    verbose=True
)

Link: 'https://www.tanzsportkreis-sankt-augustin.de/event/tsk-turnierwochenende-lm-nrw/' => SAVED                                                     


In [7]:
new_links

['https://www.tanzsportkreis-sankt-augustin.de/2023/12/13/adventliche-tanzparty-mit-pokal-turnier-und-show-2/',
 'https://www.tanzsportkreis-sankt-augustin.de/2023/12/04/adventliche-tanzparty-mit-pokal-turnier-und-show/',
 'https://www.tanzsportkreis-sankt-augustin.de/unser-club/turnierpaare/',
 'https://www.tanzsportkreis-sankt-augustin.de/veranstaltungen/turnierergebnisse/',
 'https://www.tanzsportkreis-sankt-augustin.de/veranstaltungen/competition/',
 'https://www.tanzsportkreis-sankt-augustin.de/event/tsk-turnierwochenende-lm-nrw/']

In [8]:
all_links

['https://www.tanzsportkreis-sankt-augustin.de/2023/12/13/adventliche-tanzparty-mit-pokal-turnier-und-show-2/',
 'https://www.tanzsportkreis-sankt-augustin.de/2023/12/04/adventliche-tanzparty-mit-pokal-turnier-und-show/',
 'https://www.tanzsportkreis-sankt-augustin.de/unser-club/turnierpaare/',
 'https://www.tanzsportkreis-sankt-augustin.de/veranstaltungen/turnierergebnisse/',
 'https://www.tanzsportkreis-sankt-augustin.de/veranstaltungen/competition/',
 'https://www.tanzsportkreis-sankt-augustin.de/event/tsk-turnierwochenende-lm-nrw/']

## 3. Next, identify the tournament sites and crawl for the individual competitions
Here we only crawl for results from the software `TopTurnier`. Thus the *tournament sites* and the *competition sites* can be identified by the key content:

    `<meta name="GENERATOR" content="TopTurnier">`

A competition site is found, on top of the content keyword its url ends with the anchor `\index.hmt`.

In [10]:
# Crawl competition links from a tournament link (give in url_hints.csv)
for u in all_links:
    all_tournament_links: List[str] = []
    new_tournament_links, all_tournament_links = wc.crawl_href_links_on_webpage(
        url=u,
        website_contains_content_all=[c.FindTournamentsDf.KEY_CONTENT],
        forbidden_url_prefixes=c.FindTournamentsDf.BAD_LINK_PREFIXES,
        forbidden_url_postfixes=c.FindTournamentsDf.BAD_LINK_POSTFIXES,
        verbose=True    
    )
    print(f"From site '{u}' got:\n{new_tournament_links}")


Link: 'https://www.tanzsportkreis-sankt-augustin.de/tanzen-im-tsk/breitensport/' => Already known                                                      => Already known => Already known => Already known
From site 'https://www.tanzsportkreis-sankt-augustin.de/2023/12/13/adventliche-tanzparty-mit-pokal-turnier-und-show-2/' got:
[]
Link: 'https://www.tanzsportkreis-sankt-augustin.de/tanzen-im-tsk/breitensport/' => Already known                                                     
From site 'https://www.tanzsportkreis-sankt-augustin.de/2023/12/04/adventliche-tanzparty-mit-pokal-turnier-und-show/' got:
[]
Link: 'https://www.tanzsportkreis-sankt-augustin.de/unser-club/trainer/' => Already known                                                             
From site 'https://www.tanzsportkreis-sankt-augustin.de/unser-club/turnierpaare/' got:
[]
Link: 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2023/23-08-12_TSK_Sommerturniere/' => Already known                                        
F

KeyboardInterrupt: 

In [ ]:
# TODO: DO NOT USE WEBSITE CRAWLER TO MAINTAIN THE DATAFRAMES! Do loading and saving in the classes for the Dfs. 
# TODO: Construct the maintnenace pieline to creep the dataframes upto date (including later column updates)

In [ ]:
all_tournament_links

['https://ergebnisse.tanzsportkreis-sankt-augustin.de/2022/22-08-07_TSK_Sommerturniere_sen',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2023/23-12-16_17 TSK CFP',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2015/TSK_220215/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2018/TSK_271018/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2023/23-08-12_TSK_Sommerturniere/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2016/TSK_291016/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2014/TSK_220214/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2021/TSK_111221/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2023/23-09-16_TSK_SOLO Turniere und LM NRW',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2022/22-03-19_TSK_TnwPokal',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2014/TSK_230214/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2015/TSK_281115/']

In [ ]:
    # comp_links = []
    # for tournament_link in tmp_tournament_links:
    #     if tournament_link.endswith(c.CompetitionsDf.KEY_URL_ANCHOR):
    #         comp_links.append(tournament_link)
    #         continue
    #     else:
    #         wc_comp = WebCrawler(tournament_link)
    #         potential_competition_links = wc.crawl_all_anchor_sites()
    #         competition_links = [comp_link for comp_link in potential_competition_links if comp_link.endswith(c.CompetitionsDf.KEY_URL_ANCHOR)]
    #         print(f"Found competition sites in {tournament_link}:\n{competition_links}")

In [ ]:
# Crawl competition links from a tournament link (give in url_hints.csv)
tournament_list_link = 'https://www.tanzsportkreis-sankt-augustin.de/veranstaltungen/turnierergebnisse/'
wc = WebCrawler(tournament_list_link)
new_tournament_links, all_tournament_links = wc.crawl_href_links_on_webpage([tournament_list_link], website_contains_content_all=[c.FindTournamentsDf.KEY_CONTENT], verbose=True)
all_tournament_links

Link: #2014          30.online.de/kontakt/datenschutz/          en/turnierergebnisse/           content                    ontent          desired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content           => Undesired content         

['https://ergebnisse.tanzsportkreis-sankt-augustin.de/2022/22-08-07_TSK_Sommerturniere_sen',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2023/23-12-16_17 TSK CFP',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2015/TSK_220215/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2018/TSK_271018/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2023/23-08-12_TSK_Sommerturniere/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2016/TSK_291016/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2014/TSK_220214/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2021/TSK_111221/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2023/23-09-16_TSK_SOLO Turniere und LM NRW',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2022/22-03-19_TSK_TnwPokal',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2014/TSK_230214/',
 'https://ergebnisse.tanzsportkreis-sankt-augustin.de/2015/TSK_281115/']

In [ ]:
# TODO: For all these tournaments - decide if tournament or competition. If competition: Save with base url to competition dataframe.
# If tournament, crawl all competitions and save to competition dataframe with tournament link as base.

## Test the already finished features

In [ ]:
url_hints = c.UrlHints()
url_hints.df

,Url,Scope,Federal state,Crawl date
0,https://www.tanzsportkreis-sankt-augustin.de/v...,Local,Nordrhein-Westfalen,2023-12-29
1,https://ttcrotgoldkoeln.de/turniere/,Local,Nordrhein-Westfalen,2023-12-29
2,https://www.tscbruehl.de/index.php/service/tur...,Local,Nordrhein-Westfalen,2023-12-29
3,https://askania-tsc.de/turnierergebnisse/,Local,Nordrhein-Westfalen,2023-12-29
4,https://ltv-berlin.de/de/sport/leistungssport/...,Regional,Berlin,2023-12-29
5,https://tnw.de/sport/standard-latein/turnierer...,Regional,Nordrhein-Westfalen,2023-12-29
6,https://www.tbw.de/home/service/ergebnis-archiv,Regional,Baden-Wuerttemberg,2023-12-29
7,https://ltvb.de/sport/leistungssport/ergebniss...,Regional,Bayern,2023-12-29
8,https://www.hatv.de/index.php/sport/sport-stan...,Regional,Hamburg,2023-12-29
9,https://htv.de/veranstaltungen/meisterschaften...,Regional,Hessen,2023-12-29


In [ ]:
# Given a club website, crawl for all subsites.
# These subsites are then tested, if they link to tournament sites. If this is the case, save them in the 'url_hints" list.
wc = WebCrawler('https://www.tanzsportkreis-sankt-augustin.de/', "results/tanzsportkreis-sankt-augustin/known_hits.npy", "results/tanzsportkreis-sankt-augustin/negatives.npy")
# club_subsites = wc.crawl_links_to_subsites()
# club_subsites

In [ ]:
club_tournament_site = url_hints.df[url_hints.cURL][0]
wc_competition_finder = WebCrawler(club_tournament_site)
# TODO: PEN AND PAPER WORKFLOW AND USE CASE

## Use the `UrlHints` table to collect promising URLs to search for competition sites

In [ ]:
wc_clubsites = WebCrawler(url_hints.get_club_hint_df()[url_hints.cURL][0])
club_site_candidates = wc_clubsites.crawl_href_links_on_webpage([wc_clubsites.base_url])
club_site_candidates

Processing URLs:   0%|          | 0/1 [00:00<?, ?URL/s]

Error: On site https://tnw.de/verband/vereine/, could not process link 'https://www.gruen-weiss-aachen.de'
'in <string>' requires string as left operand, not list
Error: On site https://tnw.de/verband/vereine/, could not process link 'https://tnw.de/verband/vereine//http://www.tanzen-hamm.de/'
'in <string>' requires string as left operand, not list
Error: On site https://tnw.de/verband/vereine/, could not process link 'https://tnw.de/verband/vereine//tel:+492037381669'
'in <string>' requires string as left operand, not list
Error: On site https://tnw.de/verband/vereine/, could not process link 'https://tnw.de/verband/vereine//http://die-residenz.dance/'
'in <string>' requires string as left operand, not list
Error: On site https://tnw.de/verband/vereine/, could not process link 'https://tnw.de/category/sport/standard-latein/'
'in <string>' requires string as left operand, not list
Error: On site https://tnw.de/verband/vereine/, could not process link 'https://tnw.de/verband/vereine//ht

Processing URLs:   0%|          | 0/1 [00:26<?, ?URL/s]


KeyboardInterrupt: 

In [ ]:
url_hints.get_local_urls_list()

['https://www.tanzsportkreis-sankt-augustin.de/veranstaltungen/turnierergebnisse/',
 'https://ttcrotgoldkoeln.de/turniere/',
 'https://www.tscbruehl.de/index.php/service/turnierergebnisse',
 'https://askania-tsc.de/turnierergebnisse/']

In [ ]:
url_hints.get_regional_urls_list()

['https://ltv-berlin.de/de/sport/leistungssport/ergebnisse',
 'https://tnw.de/sport/standard-latein/turnierergebnisse/',
 'https://www.tbw.de/home/service/ergebnis-archiv',
 'https://ltvb.de/sport/leistungssport/ergebnisse/?currentyear=2023',
 'https://www.hatv.de/index.php/sport/sport-standard-latein/sport-standard-latein-ergebnisse-hatv',
 'https://htv.de/veranstaltungen/meisterschaften/ergebnisarchiv/',
 'https://www.trp-tanzen.org/index.php/ergebnisse.html',
 'https://www.tanzen-slt.de/public/sport/turnierergebnisse/landesmeisterschaften.html',
 'https://www.faszination-tanzen.de/sportwelten-standard-latein.html',
 'https://www.ttsv-tanzen.de/turnierergebnisse/']

In [ ]:
url_hints.get_national_urls_list()

['https://https://www.tanzsport.de/de/sportwelt/ergebnisse']

In [ ]:
crawl_competiton_links_from_tournament_link(url_hints.get_local_urls_list()[0])

Site (1/1, 100%)>>Links (70/70, 69%)->https://www.tanzsportkreis-sankt-augustin.de/veranstaltungen/competiti...	(Found 0 new)

[]

### Append to the `Url_hints` table by crawling for local club sites.
Note that the `regional` and `national` will probably stay complete as it is (if the structure of federal states does not change).

In [ ]:
club_hints = url_hints.get_club_hint_df()
club_hints

# TODO: Load the club hints and crawl for new websites


# TODO: add a method to manually ad a new url and its corresponding federal state

# for url in url_nationals:
#     crawl_links_to_keyword_sites(
#         base_url=url, 
#         content_keyword=c.C_crawler.KEY_CONTENT, 
#         bad_link_prefixes=c.C_crawler.BAD_LINK_PREFIXES, 
#         bad_link_postfixes=c.C_crawler.BAD_LINK_POSTFIXES
#     )

,Url,Federal state,Crawl date
0,https://tnw.de/verband/vereine/,Nordrhein-Westfalen,2023-12-29
1,https://www.tbw.de/home/verband/vereine?tx_pii...,Baden-Wuerttemberg,2023-12-29
2,https://ltv-berlin.de/de/verband/vereine/verei...,Berlin,2023-12-29
3,https://tanzsport-brandenburg.de/?page_id=392,Brandenburg,2023-12-29
4,https://ltvbremen.de/verband/vereine,Bremen,2023-12-29
5,https://www.hatv.de/index.php/verband/verband-...,Hamburg,2023-12-29
6,https://htv.de/vereine/,Hessen,2023-12-29
7,https://www.tanzsport-mv.de/der-verband/,Mecklenburg-Vorpommern,2023-12-29
8,https://www.tanzen-slt.de/public/verband/die-v...,Sachsen-Anhalt,2023-12-29
9,https://www.faszination-tanzen.de/vereine-im-l...,Sachsen,2023-12-29
